In [1]:
import numpy as np
import pandas as pd
from sklearn import tree

In [8]:
train = pd.read_csv("../Data/RFFeatureGen.csv",header=0)
y_train = train['y']
X_train = train.drop('y',axis=1)

In [34]:
def traverseTree(i, children_left, children_right, feature, prev_feature):
    if feature[i] < 0:
        return
    if prev_feature is not None:
        inter_var.append(sorted([prev_feature, feature[i]]))
    traverseTree(children_left[i], children_left, children_right, feature, feature[i])
    traverseTree(children_right[i], children_left, children_right, feature, feature[i])

In [35]:
inter_var = []
for i in range(100):
    X_train_sample = X_train.sample(frac=0.2)
    y_train_sample = y_train[X_train_sample.index]
    X = X_train_sample.as_matrix()
    y = y_train_sample.tolist()
    dtr = tree.DecisionTreeRegressor(max_features=0.5,max_depth=10)
    dtr.fit(X,y)
    n_nodes = dtr.tree_.node_count
    children_left = dtr.tree_.children_left
    children_right = dtr.tree_.children_right
    feature = dtr.tree_.feature
    traverseTree(0, children_left, children_right, feature, None)

In [37]:
inter_var_dic = {}
for l in inter_var:
    if len(l) == len(set(l)):
        if tuple(l) in inter_var_dic:
            inter_var_dic[tuple(l)] += 1
        else:
            inter_var_dic[tuple(l)] = 1

In [39]:
result = sorted(inter_var_dic.items(), key=lambda x:x[1], reverse=True)

In [44]:
f = open("inter_vars.txt", "w")
cols = X_train.columns
for tp in result:
    if tp[1] < 60:
        break
    inter_var = tp[0]
    line = ""
    for var in inter_var:
        line += cols[var]+'_'
    line = line[:-1]
    line += "\n"
    f.write(line)
f.close()